In [1]:
# Find path to PySpark.
import findspark
findspark.init("<Spark folder path>")

# Import PySpark and initialize SparkContext object.
import pyspark
from pyspark.sql import SQLContext


sc = pyspark.SparkContext()
sqlCtx = SQLContext(sc)


In [ ]:
#Load the data files and count # of records to verify that whole data is loaded

In [86]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

#df = sqlCtx.read.text("/Volumes/JetDrive/Root/Datasets/Datathon/*.txt")
data_RDD1 = sc.textFile("/Volumes/JetDrive/Root/Datasets/Datathon/Patients/*.txt")
#df = sqlCtx.read.text("/Volumes/JetDrive/Root/Datasets/Datathon/patients_1.txt")
#tables = df.registerTempTable('patient1')
#df2 = sqlCtx.sql("describe patient1")
#df2.show(5)
splitData_RDD1 = data_RDD1.map(lambda line: line.split('\t'))
splitData_RDD2 = splitData_RDD1.filter(lambda line : "Patient_ID" not in line)
schemaData = StructType([ \
    StructField("Patient_ID", StringType(), False),\
    StructField("Store_ID", StringType(), True),\
    StructField("Prescriber_ID", StringType(), True),\
    StructField("Drug_ID", StringType(), True),\
    StructField("SourceSystem_Code", StringType(), True),\
    StructField("Prescription_Week", StringType(), True),\
    StructField("Dispense_Week", StringType(), True),\
    StructField("Drug_Code", StringType(), True),\
    StructField("NHS_Code", StringType(), True),\
    StructField("IsDeferredScript", StringType(), True),\
    StructField("Script_Qty", StringType(), True),\
    StructField("Dispensed_Qty", StringType(), True),\
    StructField("MaxDispense_Qty", StringType(), True),\
    StructField("PatientPrice_Amt", StringType(), True),\
    StructField("WholeSalePrice_Amt", StringType(), True),\
    StructField("GovernmentReclaim_Amt", StringType(), True),\
    StructField("RepeatsTotal_Qty", StringType(), True),\
    StructField("RepeatsLeft_Qty", StringType(), True),\
    StructField("StreamlinedApproval_Code", StringType(), True)])

#sorted_fields = sorted(schema.fields, key=lambda x: (int(x[0])))
#sorted_fields = schema.map(lambda p: (int(p[0]))) 
#sorted_schema = StructType(fields=sorted_fields)

data_DF = sqlCtx.createDataFrame(splitData_RDD2, schemaData)
tables = data_DF.registerTempTable('transactions')



data_DF2 = sqlCtx.sql('select Patient_ID,Store_ID,Prescriber_ID,Drug_ID,Prescription_Week from transactions where Patient_ID = \
                 4 limit 5')

data_DF2.show()


+----------+--------+-------------+-------+-----------------+
|Patient_ID|Store_ID|Prescriber_ID|Drug_ID|Prescription_Week|
+----------+--------+-------------+-------+-----------------+
|         4|    2143|        40822|   4867|       2011-08-07|
|         4|    2143|        40822|   5476|       2011-05-01|
|         4|    2143|        40822|   4867|       2011-08-07|
|         4|    2143|        40822|     99|       2011-09-18|
|         4|    2143|        40822|     99|       2011-09-18|
+----------+--------+-------------+-------+-----------------+



In [87]:
patients_RDD = sc.textFile("/Volumes/JetDrive/Root/Datasets/Datathon/Lookups/patients.txt")
splitPatients_RDD1 = patients_RDD.map(lambda line: line.split('\t'))
splitPatients_RDD2 = splitPatients_RDD1.filter(lambda line : "Patient_ID" not in line)

schemaPatients = StructType([ \
    StructField("Patient_ID", StringType(), False),\
    StructField("gender", StringType(), True),\
    StructField("year_of_birth", StringType(), True),\
    StructField("postcode", StringType(), True)])
 
patients_DF = sqlCtx.createDataFrame(splitPatients_RDD2, schemaPatients)

tables2 = patients_DF.registerTempTable('patients')

patients_DF2 = sqlCtx.sql('select Patient_ID,gender,year_of_birth,postcode from patients where Patient_ID > 4 limit 5')

patients_DF2.show()


+----------+------+-------------+--------+
|Patient_ID|gender|year_of_birth|postcode|
+----------+------+-------------+--------+
|         5|     U|         1900|    2229|
|         6|     U|         1900|    3162|
|         7|     M|         1900|    6150|
|         8|     U|         1900|    4306|
|         9|     F|         1900|    6020|
+----------+------+-------------+--------+



In [88]:
joined_DF3 = sqlCtx.sql('select transactions.Patient_ID, gender, year_of_birth, Prescription_Week from transactions \
                        join patients on transactions.Patient_ID = patients.Patient_ID limit 4')
joined_DF3.show()


+----------+------+-------------+-----------------+
|Patient_ID|gender|year_of_birth|Prescription_Week|
+----------+------+-------------+-----------------+
|    100010|     F|         1900|       2011-01-30|
|    100010|     F|         1900|       2011-01-30|
|    100010|     F|         1900|       2011-01-30|
|    100010|     F|         1900|       2011-07-10|
+----------+------+-------------+-----------------+

